In [ ]:
# IDEA: use ProtTrans pipeline as feature extraction (instead of embeddings) and build model on top of it
# check whether input format makes sense
# https://github.com/agemagician/ProtTrans/blob/master/Embedding/PyTorch/Basic/ProtBert.ipynb

In [ ]:
import os
import sys
import csv
import pickle
import random
import numpy as np
from time import time
from tqdm import tqdm

from typing import List, Tuple

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from torchtext.vocab import build_vocab_from_iterator

In [ ]:
def seed_everything(seed: int):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
seed_everything(1234)

In [ ]:
# def read_data(path):
#     with open(path, 'r') as csvfile:
#         train_data = list(csv.reader(csvfile))[1:] # skip col name
#         sents, lbls = [], []
#         for i in range(0, len(train_data), 16):
#             s, l = zip(*train_data[i:i+16])
#             sents.append(s)
#             lbls.append(l)
#     return sents, lbls

def read_data(path):
    with open(path, 'r') as csvfile:
        train_data = list(csv.reader(csvfile))[1:] # skip col name
        sents, lbls = [], []
        for s, l in train_data:
            sents.append(s)
            lbls.append(l)
    return sents, lbls

# number of trainable parameters in model
def get_total_model_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
class CleavageDataset(Dataset):
    def __init__(self, seq, lbl):
        self.seq = seq
        self.lbl = lbl
    
    def __getitem__(self, idx):
        return self.seq[idx], self.lbl[idx]
    
    def __len__(self):
        return len(self.lbl)
    
class CleavageBatch:
    def __init__(self, batch: List[Tuple[str, str]]):
        ordered_batch = list(zip(*batch))
        self.bw_seq = torch.tensor([encode_text(seq)[::-1] for seq in ordered_batch[0]], dtype=torch.int64)
        self.fw_seq = torch.tensor([encode_text(seq) for seq in ordered_batch[0]], dtype=torch.int64)
        self.lbl = torch.tensor([int(l) for l in ordered_batch[1]], dtype=torch.float)
        
    def pin_memory(self):
        self.bw_seq = self.bw_seq.pin_memory()
        self.fw_seq = self.fw_seq.pin_memory()
        self.lbl = self.lbl.pin_memory()
        return self
    
def collate_wrapper(batch):
    return CleavageBatch(batch)

In [ ]:
class SeqEncoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim, rnn_size, dropout):
        super().__init__()
        
        self.fw_embedding = nn.Embedding(
            num_embeddings=vocab_size, embedding_dim=embedding_dim
        )
        
        self.bw_embedding = nn.Embedding(
            num_embeddings=vocab_size, embedding_dim=embedding_dim
        )
        
        self.dropout = nn.Dropout(dropout)
        
        self.fw_lstm = nn.LSTM(
            input_size=embedding_dim, hidden_size=rnn_size, batch_first=True
        )
        
        self.bw_lstm = nn.LSTM(
            input_size=embedding_dim, hidden_size=rnn_size, batch_first=True
        )
        
    def forward(self, bw_seq, fw_seq):
        # input shape: (batch_size, seq_len=10)
        fw_embeddings = self.dropout(self.fw_embedding(fw_seq))
        bw_embeddings = self.dropout(self.bw_embedding(bw_seq))
    
        # input shape: (batch_size, seq_len, embedding_dim)
        fw_out, _ = self.fw_lstm(fw_embeddings)
        bw_out, _ = self.bw_lstm(bw_embeddings)
        
        # input shape: (batch_size, seq_len, rnn_size)
        # only get representation at last t
        fw_out = self.dropout(fw_out[:, -1, :])
        bw_out = self.dropout(bw_out[:, -1, :])
        
        # input shape: (batch_size, rnn_size)
        # out shape: (batch_size, 2*rnn_size)
        return torch.cat([fw_out, bw_out], dim=1) 
        
    

class FwBwQuadBiLSTM(nn.Module):
    def __init__(self, vocab_size, seq_enc_emb_dim, seq_enc_rnn_size, rnn_size1, rnn_size2, rnn_size3, rnn_size4, hidden_size, dropout):
        super().__init__()
        
        # sequence encoder replaces embedding representations
        self.seq_encoder = SeqEncoder(
            vocab_size=vocab_size,
            embedding_dim=seq_enc_emb_dim,
            rnn_size=seq_enc_rnn_size,
            dropout=dropout
        )
        
        self.dropout=nn.Dropout(dropout)
        
        self.lstm1 = nn.LSTM(
            input_size=seq_enc_rnn_size * 2,
            hidden_size=rnn_size1,
            bidirectional=True,
            batch_first=True
        )
        
        self.lstm2 = nn.LSTM(
            input_size=2*rnn_size1,
            hidden_size=rnn_size2,
            bidirectional=True,
            batch_first=True
        )
        
        self.lstm3 = nn.LSTM(
            input_size=2*rnn_size2,
            hidden_size=rnn_size3,
            bidirectional=True,
            batch_first=True
        )
        
        self.lstm4 = nn.LSTM(
            input_size=2*rnn_size3,
            hidden_size=rnn_size4,
            bidirectional=True,
            batch_first=True
        )
        
        self.fc1 = nn.Linear(rnn_size4 * 2, hidden_size)
        self.fc2 = nn.Linear(hidden_size, 1)
        
    def forward(self, bw_seq, fw_seq):
        # input shape: (batch_size, seq_len=10)
        embedded = self.dropout(self.seq_encoder(bw_seq, fw_seq))
        
        # input shape: (batch_size, seq_enc_rnn_size * 2)
        out, _ = self.lstm1(embedded)
        
        # input shape: (batch_size, 2*rnn_size)
        out, _ = self.lstm2(out)
        
        # input shape: (batch_size, 2*rnn_size)
        out, _ = self.lstm3(out)
        
        # input shape: (batch_size, 2*rnn_size)
        out, _ = self.lstm4(out)
        
        # input shape; (batch_size, 2*rnn_size)
        out = self.dropout(F.relu(self.fc1(out)))
        
        # input shape: (batch_size, hidden_size)
        # output shape: (batch_size)
        return self.fc2(out).squeeze()

In [ ]:
def process(model, loader, criterion, optim=None):
    epoch_loss, num_correct, total = 0, 0, 0
    
    for batch in tqdm(
        loader,
        desc="Train: " if optim is not None else "Eval: ",
        file=sys.stdout,
        unit="batches"
    ):
        bw_seq, fw_seq, lbl = batch.bw_seq, batch.fw_seq, batch.lbl
        bw_seq, fw_seq, lbl = bw_seq.to(device), fw_seq.to(device), lbl.to(device)
        
        scores = model(bw_seq, fw_seq)
        loss = criterion(scores, lbl)
        
        if optim is not None:
            optim.zero_grad()
            loss.backward()
            optim.step()
        
        epoch_loss += loss.item()
        num_correct += ((scores > 0) == lbl).sum()
        total += len(lbl)
    return epoch_loss / total, num_correct / total

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# load train and dev data
train_seqs, train_lbl = read_data('../../data/n_train.csv')
dev_seqs, dev_lbl = read_data('../../data/n_val.csv')

# create vocab from train seqs
vocab = build_vocab_from_iterator(train_seqs, specials=['<UNK>'])
vocab.set_default_index(vocab['<UNK>'])
encode_text = lambda x: vocab(list(x))

In [ ]:
NUM_EPOCHS = 10
BATCH_SIZE = 512
VOCAB_SIZE = len(vocab)
SEQ_ENC_EMB_DIM = 100
SEQ_ENC_RNN_SIZE = 200
RNN_SIZE1 = 128
RNN_SIZE2 = 512
RNN_SIZE3 = 256
RNN_SIZE4 = 128
HIDDEN_SIZE = 128
DROPOUT = 0.5
LEARNING_RATE = 1e-4

model = FwBwQuadBiLSTM(
    vocab_size=VOCAB_SIZE,
    seq_enc_emb_dim=SEQ_ENC_EMB_DIM,
    seq_enc_rnn_size=SEQ_ENC_RNN_SIZE,
    rnn_size1=RNN_SIZE1,
    rnn_size2=RNN_SIZE2,
    rnn_size3=RNN_SIZE3,
    rnn_size4=RNN_SIZE4,
    hidden_size=HIDDEN_SIZE,
    dropout=DROPOUT
).to(device)

optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
criterion = nn.BCEWithLogitsLoss()

# create train and dev loader
train_data = CleavageDataset(train_seqs, train_lbl)
train_loader = DataLoader(train_data, batch_size = BATCH_SIZE, shuffle=True, collate_fn=collate_wrapper, pin_memory=True, num_workers=10)

dev_data = CleavageDataset(dev_seqs, dev_lbl)
dev_loader = DataLoader(dev_data, batch_size = BATCH_SIZE, shuffle=True, collate_fn=collate_wrapper, pin_memory=True, num_workers=10)

print(f"Total trainable model parameters: {get_total_model_params(model):,}")

In [ ]:
start = time()
print("Starting Training.")
highest_val_acc = 0
train_losses, train_accuracies= [], []
val_losses, val_accuracies = [], []

for epoch in range(1, NUM_EPOCHS + 1):
    model.train()
    train_loss, train_acc = process(model, train_loader, criterion, optimizer)
    
    model.eval()
    with torch.no_grad():
        val_loss, val_acc = process(model, dev_loader, criterion)
        
    # save current acc, loss
    train_losses.append((epoch, train_loss))
    train_accuracies.append((epoch, train_acc))
    val_losses.append((epoch, val_loss))
    val_accuracies.append((epoch, val_acc))
    
    if val_acc > highest_val_acc:
        highest_val_acc = val_acc
        path = f"../../params/n_term/quadBiLSTM/fwbw_acc{val_acc:.4f}_epoch{epoch}.pt"
        torch.save(model.state_dict(), path)
        
    print(
        f"Training:   [Epoch {epoch:2d}, Loss: {train_loss:8.4f}, Acc: {train_acc:.4f}]"
    )
    print(f"Evaluation: [Epoch {epoch:2d}, Loss: {val_loss:8.4f}, Acc: {val_acc:.4f}]")
    
print("Finished Training.")
train_time = (time() - start) / 60
print(f"Training took {train_time} minutes.")

In [ ]:
# save training stats
lsts = [train_losses, train_accuracies, val_losses, val_accuracies, train_time]
names = [
    "train_losses",
    "train_accuracies",
    "val_losses",
    "val_accuracies",
    "train_time",
]
to_save = dict()
for name, lst in zip(names, lsts):
    to_save[name] = lst

with open(f"../params/n_term/quadBiLSTM/metrics.pkl", "wb") as f:
    pickle.dump(to_save, f, pickle.HIGHEST_PROTOCOL)

print("Finished Saving Details.")